In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import PartialDependenceDisplay
from matplotlib import pyplot as plt

from diamond import data
from diamond import model

In [ ]:
X_train, X_test, y_train, y_test = data.split(
    *data.get_X_y(data.clean(data.load_raw('datasets/diamonds/diamonds.csv'))))

In [ ]:
model.pipeline

In [ ]:
parameter_grid = model.make_params(model.DEFAULT_USER_PARAMETER_GRID)
model.DEFAULT_USER_PARAMETER_GRID

In [ ]:
search = GridSearchCV(model.pipeline, scoring='r2', param_grid=parameter_grid,
                      refit=True)
search.fit(X_train, y_train)
search.best_estimator_

In [ ]:
search.best_score_

In [ ]:
for params, score in zip(search.cv_results_['params'],
                         search.cv_results_['mean_test_score']):
    print(params, score)

In [ ]:
print(search.best_params_)
print(search.best_estimator_.named_steps['linear'].regressor_.coef_)
print(search.best_estimator_.named_steps['linear'].regressor_.intercept_)

In [ ]:
X_trans = data.sequential_encoder.fit_transform(X_train)

_, ax = plt.subplots(figsize=(10, 7))
PartialDependenceDisplay.from_estimator(search.best_estimator_[1:], X_trans,
                                        ('carat', 'cut', 'color', 'clarity'),
                                        kind='both', centered=True, ax=ax,
                                        n_cols=2)
plt.show()